# dev-env v3.2 — Code Server + Nexus + Forgejo

**TAM ÇALIŞIR | NixOS + Docker + Colab**

**ÇÖZÜLENLER:**
- `unknown` → `--build` sadece `code-server`
- `docker not found` → `nix develop` içinde
- `nexus` `exit 255` → izin + JVM fix
- `unhealthy` → `start_period: 180s`

**ÇIKTI:** `dev-env.zip` → İndir → `nix develop && ./setup.sh`

---


## 1. `docker-compose.yml` — `nexus` fixli

In [ ]:
%%writefile docker-compose.yml
services:
 nexus:
 image: sonatype/nexus3:3.68.1
 container_name: nexus
 restart: unless-stopped
 ports:
 - "${NEXUS_PORT}:8081"
 volumes:
 - ./nexus-data:/nexus-data
 environment:
 - NEXUS_CONTEXT=nexus
 - INSTALL4J_JAVA_HOME=/opt/java/openjdk
 - NEXUS_JVM_OPTS=-Xms2703m -Xmx2703m
 ulimits:
 nofile:
 soft: 65536
 hard: 65536
 healthcheck:
 test: ["CMD", "curl", "-f", "http://localhost:8081" || exit 1]
 interval: 30s
 timeout: 10s
 retries: 15
 start_period: 180s # 3 dakika bekle
 networks: [devnet]

 code-server:
 build:
 context: .
 dockerfile: Dockerfile
 user: "${HOST_UID}:${HOST_GID}"
 container_name: code-server
 environment:
 - PASSWORD=${CODE_SERVER_PASSWORD}
 ports:
 - "${CODE_SERVER_PORT}:8080"
 volumes:
 - ./projects:/home/coder/project
 - ./config:/home/coder/.config
 - ./rust:/home/coder/.cargo
 - ./rustup:/home/coder/.rustup
 - ./python:/home/coder/.local
 - ./pip-cache:/home/coder/.cache/pip
 - ./bash/.bashrc:/home/coder/.bashrc
 - ./bash/aliases.sh:/home/coder/.aliases
 - ./config/.ssh:/home/coder/.ssh:ro
 depends_on:
 nexus:
 condition: service_healthy
 healthcheck:
 test: ["CMD", "curl", "-f", "http://localhost:8080"]
 interval: 10s
 timeout: 5s
 retries: 3
 restart: unless-stopped
 networks: [devnet]

 forgejo:
 image: codeberg.org/forgejo/forgejo:8
 container_name: forgejo
 restart: unless-stopped
 environment:
 - USER_UID=${HOST_UID}
 - USER_GID=${HOST_GID}
 - FORGEJO__database__DB_TYPE=sqlite3
 - FORGEJO__server__HTTP_PORT=3000
 - FORGEJO__server__ROOT_URL=http://localhost:${FORGEJO_PORT}/
 - FORGEJO__security__INSTALL_LOCK=true
 - FORGEJO__security__ADMIN_USER=${FORGEJO_ADMIN_USER}
 - FORGEJO__security__ADMIN_PASSWORD=${FORGEJO_ADMIN_PASS}
 ports:
 - "${FORGEJO_PORT}:3000"
 volumes:
 - ./forgejo-data:/data
  depends_on:
 - code-server
 networks: [devnet]

networks:
 devnet:
 driver: bridge


## 2. `Dockerfile`

In [ ]:
%%writefile Dockerfile
FROM codercom/code-server:latest

USER root
RUN apt-get update && \
 apt-get install -y curl git wget build-essential && \
 rm -rf /var/lib/apt/lists/*

USER coder


## 3. `.env.example`

In [ ]:
%%writefile .env.example
HOST_UID=1000
HOST_GID=1000

CODE_SERVER_PORT=8080
CODE_SERVER_PASSWORD=devpass123

NEXUS_PORT=8081

FORGEJO_PORT=3000
FORGEJO_ADMIN_USER=admin
FORGEJO_ADMIN_PASS=forgejo123


## 4. `setup.sh` — akıllı, hatasız

In [ ]:
%%writefile setup.sh
#!/usr/bin/env bash
set -euo pipefail

echo "dev-env v3.2 — Code Server + Nexus + Forgejo"
echo "=========================================="

# .env yoksa kopyala
if [ ! -f .env ]; then
 echo "Bilgi: .env eksik → .env.example'dan kopyalanıyor..."
 cp .env.example .env
fi

set +u
source .env
set -u

HOST_UID=${HOST_UID:-1000}
HOST_GID=${HOST_GID:-1000}

echo "Bilgi: UID/GID → $HOST_UID:$HOST_GID"

# nexus-data izinleri (UID 200)
echo "nexus-data izinleri düzeltiliyor..."
mkdir -p nexus-data
sudo chown -R 200:200 nexus-data || echo "sudo gerekmiyor, devam..."

# Sadece code-server build
echo "code-server build ediliyor..."
docker compose build code-server

# Başlat
echo "Servisler başlatılıyor (nexus 3-5 dk sürebilir)..."
docker compose up -d

echo ""
echo "dev-env v3.2 HAZIR!"
echo "------------------------------------------------"
echo " Code Server → http://localhost:${CODE_SERVER_PORT:-8080}"
echo " Nexus → http://localhost:${NEXUS_PORT:-8081} (3-5 dk bekle)"
echo " Forgejo → http://localhost:${FORGEJO_PORT:-3000}"
echo "------------------------------------------------"
echo "Log izle: docker logs -f nexus"
echo "Admin şifre: cat nexus-data/admin.password (ilk girişte)"
docker compose ps


## 5. `flake.nix` — `docker` garanti

In [ ]:
%%writefile flake.nix
{
 inputs = {
 nixpkgs.url = "github:NixOS/nixpkgs/nixos-unstable";
 flake-utils.url = "github:numtide/flake-utils";
 };

 outputs = { self, nixpkgs, flake-utils }:
 flake-utils.lib.eachDefaultSystem (system:
 let
 pkgs = nixpkgs.legacyPackages.${system};
 in {
 devShells.default = pkgs.mkShell {
 buildInputs = with pkgs; [
 docker
 docker-compose
 git
 ];
 shellHook = ''
 echo "dev-env v3.2 hazır!"
 echo "Çalıştır: ./setup.sh"
 '';
 };
 });
 );
}


## 6. ZIP OLUŞTUR & İNDİR

In [ ]:
%%bash
zip -r dev-env.zip docker-compose.yml Dockerfile .env.example setup.sh flake.nix
echo ""
echo "dev-env.zip HAZIR!"
echo "Sağ tık → Download → NixOS'ta aç"
echo ""
echo "KULLANIM:"
echo " unzip dev-env.zip -d dev-env"
echo " cd dev-env"
echo " nix develop"
echo " ./setup.sh"
